un modelo entrenado con los reviews de IMDB al que yo le paso un tweet de una pelicula y me diga que rating le hubiera colocado (del 1 al 5)

#aca comienza

In [ ]:
from json import loads
import pandas as pd
from datetime import date
!pip install Cinemagoer               #API de IMDb pero solo muestra las primeras 25 reviews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 301 kB 8.6 MB/s 


In [ ]:
from imdb import Cinemagoer
meses = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'] #lo usaré después

# create an instance of the Cinemagoer class
ia = Cinemagoer()

top = ia.get_popular100_movies()
top100IDs = []

for i in range(len(top)):
  top100IDs.append(top[i].movieID)

print(top100IDs)

['9783600', '1745960', '10298810', '9419884', '3704428', '8041270', '0092099', '7144666', '8009428', '1655389', '10648342', '6710474', '14549466', '13249596', '13352968', '6467266', '12004038', '11671006', '11138512', '1877830', '8323668', '11291274', '0068646', '14376344', '4123432', '13560574', '15090124', '2382320', '6455162', '14174940', '1517268', '0107290', '10872600', '5108870', '13320622', '5113044', '9179430', '12593682', '9411972', '1464335', '14814040', '13904644', '11703710', '10886166', '4881806', '7657566', '1396484', '8178634', '14145426', '4513678', '20256528', '12412888', '0369610', '8115900', '8550054', '8772262', '15521050', '1160419', '10954984', '7131622', '10809742', '0071562', '0076759', '15189534', '15474916', '7466810', '4998632', '10954652', '9639470', '9731598', '14043966', '11214590', '5315212', '3513500', '0111161', '3447590', '14349032', '9032400', '12996154', '11196036', '15004136', '14439896', '13139228', '0120915', '13841850', '15218000', '4154796', '12

In [ ]:
#si la fecha es un string, debo enotnces separar en dia mes y year. Y para ordenar por fecha de release, asi obtener las 10 mas nuevas

#es una funcion a la que le paso la lista de IDs del top 100 y lo que hace es hacer un DF con ID, Nombre, PosterURL, dia, mes, año, y cantDeVotos
def dameTop100IDsPorFecha(listaIDs, meses):

  #keys
  pelis = []
  nombres = []
  posters = []
  d = []
  m = []
  y = []
  votes = []

  for id in listaIDs:
    movie = ia.get_movie(id)
    #Siempre y cuando la pelicula en el API tenga fecha de release y cantDeVotos, hace el procedimiento de:
    if(('original air date' and 'votes') in list(movie.keys())):
        #Obtener titulo, votos, posterUrl y fecha
        name = movie['title']
        v = movie['votes']
        poster = movie['full-size cover url']
        fecha = movie['original air date'][:11]
        print(name)
        print(fecha)

        pelis.append(id)  #movie)
        nombres.append(name)
        votes.append(v)
        posters.append(poster)

        if(fecha[2] == ' ' and fecha[6]==' '):
          d.append(int(fecha[:2]))
          m.append(int(meses.index(fecha[3:6]) + 1))
          y.append(int(fecha[7:11]))
        elif fecha[3] == ' ':
          d.append(0)
          m.append(int(meses.index(fecha[:3]) + 1))
          y.append(int(fecha[4:8]))
        else:
          temp = "".join([ele for ele in fecha if ele.isdigit()])
          d.append(0)
          m.append(0)
          y.append(int(temp))

    else:
      listaIDs.pop(listaIDs.index(id))

    #Para luego armar un DF con todos estos datos
    dic = {'pelisIDs': pelis, 'nombres': nombres, 'dia':d, 'mes':m, 'year': y, 'votos':votes, 'posters':posters}
    df = pd.DataFrame.from_dict(dic)

  return df

In [ ]:
df = dameTop100IDsPorFecha(top100IDs, meses)

Spiderhead
17 Jun 2022
Top Gun: Maverick
27 May 2022
Lightyear
17 Jun 2022
Doctor Strange in the Multiverse of Madness
06 May 2022
Elvis
24 Jun 2022
Jurassic World Dominion
10 Jun 2022
Top Gun
16 May 1986
The Black Phone
24 Jun 2022
Hustle
08 Jun 2022
Everything Everywhere All at Once
25 Mar 2022
Crimes of the Future
25 May 2022
Father of the Bride
16 Jun 2022
Good Luck to You, Leo Grande
17 Jun 2022
Sing 2
22 Dec 2021
Watcher
03 Jun 2022
The Man from Toronto
24 Jun 2022
The Northman
22 Apr 2022
The Batman
04 Mar 2022
Jerry and Marge Go Large
17 Jun 2022
The Unbearable Weight of Massive Talent
22 Apr 2022
The Godfather
24 Mar 1972
Cha Cha Real Smooth
17 Jun 2022
Fantastic Beasts: The Secrets of Dumbledore
15 Apr 2022
X
18 Mar 2022
Mad God
16 Jun 2022
No Time to Die
08 Oct 2021
Bhool Bhulaiyaa 2
19 May 2022
Interceptor
03 Jun 2022
Spider-Man: No Way Home
17 Dec 2021
Morbius
01 Apr 2022
The Lost City
25 Mar 2022
Minions: The Rise of Gru
01 Jul 2022
Vikram
2022 (Brazi
Uncharted
18 Feb 202

In [ ]:
#Luego ordeno ese DF por fecha
dfOrdenado = df.sort_values(by=['year', 'mes', 'dia'], ascending=[False, False, False], ignore_index = True)
#Para que asi entre menor sea el número de fila, significa que más reciente es esa pelicula
dfOrdenado.head()

,pelisIDs,nombres,dia,mes,year,votos,posters
0,5113044,Minions: The Rise of Gru,1,7,2022,2661,https://m.media-amazon.com/images/M/MV5BZDQyOD...
1,3704428,Elvis,24,6,2022,19621,https://m.media-amazon.com/images/M/MV5BMmUzMT...
2,7144666,The Black Phone,24,6,2022,15638,https://m.media-amazon.com/images/M/MV5BOWVmNT...
3,11671006,The Man from Toronto,24,6,2022,14016,https://m.media-amazon.com/images/M/MV5BZDM2Yj...
4,14145426,Beavis and Butt-Head Do the Universe,23,6,2022,4022,https://m.media-amazon.com/images/M/MV5BOTc5Zm...


In [ ]:
# Con este codigo obtengo la fecha de hoy
today = date.today()
dHoy = today.day
mHoy = today.month
yHoy = today.year
print('Hoy es: ' + str(dHoy) + '/' + str(mHoy) + '/' + str(yHoy))

Hoy es: 1/7/2022


In [ ]:
#"popular" = más de 100.000 votos en IMDb
#busco MAS populares y MAS recientes(pero que YA se haya estrenado, es decir: La fecha sea menorOIgual a la fecha de hoy)
#while sea mayor a la fecha de hoy, seguir avanzando el index

dfTop10PopYNew = dfOrdenado.loc[(dfOrdenado['votos'] > 100000)][:15]

indicesPopulares = list(dfTop10PopYNew.index)
las10MasRecientesYMasPopulares = list(dfTop10PopYNew['pelisIDs']) #Aca voy guardando las IDs de las peliculas que voy a scrapear después


# i = 0
# while(i<dfOrdenado.shape[0] and (
#     (dfOrdenado['year'].loc[i] > yHoy) or
#     (dfOrdenado['year'].loc[i] == yHoy and dfOrdenado['mes'].loc[i] > mHoy) or
#     (dfOrdenado['year'].loc[i] == yHoy and dfOrdenado['mes'].loc[i] == mHoy and dfOrdenado['dia'].loc[i] > dHoy))):
#   i+=1


# url = 'https://www.imdb.com/title/tt'
# #A partir del i obtenido en el ciclo anterior, es el indice que tiene peliculas que ya se estrenaron
# while i < dfOrdenado.shape[0] and len(las10MasRecientesYMasPopulares) < 15:
#   id = dfOrdenado['pelisIDs'].iloc[i]

#   if(dfOrdenado['votos'].iloc[i] > 100000): #siempre que tengan más de 100.000 votos
#       las10MasRecientesYMasPopulares.append(id)
#       indicesPopulares.append(i)
#       print(url + dfOrdenado['pelisIDs'].iloc[i] + '/reviews')
#       print(str(dfOrdenado['dia'].iloc[i]) + '/' + str(meses[dfOrdenado['mes'].iloc[i] -1]) + '/' + str(dfOrdenado['year'].iloc[i]))
#   i+=1


In [ ]:
print(indicesPopulares[:15])
print(las10MasRecientesYMasPopulares[:15])

dfTop10PopYNew.to_csv('infoPelisTop.csv')

[22, 29, 31, 34, 39, 43, 44, 45, 48, 49, 50, 51, 52, 53, 54]
['1745960', '9419884', '11138512', '4123432', '6710474', '1877830', '1464335', '7657566', '10872600', '11214590', '4513678', '9032400', '9639470', '1160419', '2382320']


In [ ]:
dfTop10PopYNew

,pelisIDs,nombres,dia,mes,year,votos,posters
22,1745960,Top Gun: Maverick,27,5,2022,202763,https://m.media-amazon.com/images/M/MV5BOWQwOT...
29,9419884,Doctor Strange in the Multiverse of Madness,6,5,2022,265816,https://m.media-amazon.com/images/M/MV5BNWM0ZG...
31,11138512,The Northman,22,4,2022,132303,https://m.media-amazon.com/images/M/MV5BMzVlMm...
34,4123432,Fantastic Beasts: The Secrets of Dumbledore,15,4,2022,104259,https://m.media-amazon.com/images/M/MV5BZGQ1Nj...
39,6710474,Everything Everywhere All at Once,25,3,2022,131617,https://m.media-amazon.com/images/M/MV5BYTdiOT...
43,1877830,The Batman,4,3,2022,527764,https://m.media-amazon.com/images/M/MV5BMDdmMT...
44,1464335,Uncharted,18,2,2022,139528,https://m.media-amazon.com/images/M/MV5BMWEwNj...
45,7657566,Death on the Nile,11,2,2022,121679,https://m.media-amazon.com/images/M/MV5BNjI4ZT...
48,10872600,Spider-Man: No Way Home,17,12,2021,666334,https://m.media-amazon.com/images/M/MV5BZWMyYz...
49,11214590,House of Gucci,24,11,2021,108722,https://m.media-amazon.com/images/M/MV5BYzdlMT...


opciones:

a) estaria bueno entrenar al modelo con una pelicula, y luego pasarle tweets de esa misma pelicula?

b) ooOoOOo debería entrenarlo con todas las peliculas y luego evaluar en un tweet de algunas de las pelis al azar?

Podria probar hacer un modelo con todos y un modelo con una sola peli, y ver con el test set de el modelo con una sola peli, para cual de los dos modelos obtengo mejor performance

#Run para importar cosas que se necesitan para scrap

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
from selenium import webdriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 42.8 MB/s 
     |████████████████████████████████| 358 kB 44.1 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 48.9 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://c

In [ ]:
import requests #para hacer los gets
from bs4 import BeautifulSoup as BS #ehhhhhhh aun no se
from selenium.webdriver.common.by import By

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# Instanciamos una clase que nos permite darle el seteo al driver que usaremos
options = webdriver.ChromeOptions()
# Seteamos algunas opciones para que no muera en colab
options.add_argument("start-maximized") # https://stackoverflow.com/a/26283818/1689770
options.add_argument("enable-automation") # https://stackoverflow.com/a/43840128/1689770
options.add_argument("--headless") # only if you are ACTUALLY running headless
options.add_argument("--no-sandbox") # https://stackoverflow.com/a/50725918/1689770
options.add_argument("--disable-infobars") #https://stackoverflow.com/a/43840128/1689770
options.add_argument("--disable-dev-shm-usage") # https://stackoverflow.com/a/50725918/1689770
options.add_argument("--disable-browser-side-navigation") # https://stackoverflow.com/a/49123152/1689770
options.add_argument("--disable-gpu") # https://stackoverflow.com/questions/51959986/how-to-solve-selenium-chromedriver-timed-out-receiving-message-from-renderer-exc

#scrap para todas las IDs de pelis mas recientes, mas populares y con mas de 100.000 votos

In [ ]:
#Existe abajo de cada review una linea que dice a cuantas personas les pareció
#de ayuda esa review. Está bueno obtener este parámetro también
def obtenerHelpfulness(otra):
  oracion = otra.get_attribute('textContent').lstrip().rstrip().replace(',', '')
  hasta = oracion.index(' found')
  #print(oracion[:hasta])

  posCantDeVotos = oracion.rindex(' ', 0, hasta) + 1
  cantVotos = int(oracion[posCantDeVotos:hasta])

  posCuantosPositivos = oracion.index(' ', 0, posCantDeVotos)
  cantVotosPositivos = int(oracion[0:posCuantosPositivos])

  return cantVotosPositivos, cantVotos

In [ ]:
#función que hace el scrap. "masReviews" es una lista que tiene todos los cointeners del html con cada review
#y "masHelpfulness" la lista con cada conteiner que tiene la línea que dice de que tan ayuda fue esa review
#Ambas tienen el mismo largo ya que cada review tiene ambos conteiners
def hazDFdeReviews(masReviews, masHelpfulness):

  titulos =[]
  puntajes = []
  textos = []
  ratios = []
  vPos = []
  vTotales =[]

  #por cada review obtengo:
  for i in range(len(masReviews)):
    if(i == ((len(masReviews)*25) // 100)):
      print('Va el 25% de las reviews cargadas')
    elif(i == ((len(masReviews)*50) // 100)):
      print('Va el 50% de las reviews cargadas')
    elif(i == ((len(masReviews)*75) // 100)):
      print('Va el 75% de las reviews cargadas')
    elif(i == ((len(masReviews)*95) // 100)):
      print('Va el 95% de las reviews cargadas')

    try:

      #todos estos son WebElements
      tagScore = masReviews[i].find_element(By.CLASS_NAME, 'rating-other-user-rating').find_elements(By.TAG_NAME, 'span') #lo que contiene al string con el puntaje
      tagTitle = masReviews[i].find_element(By.CLASS_NAME, 'title') #lo que contiene al string con el titulo de la review
      tagTextoReview = masReviews[i].find_element(By.CLASS_NAME, "text,show-more__control") #lo que contiene al string con el texto de la review


      # tagScore = masReviews[i].find_element_by_class_name('rating-other-user-rating').find_elements_by_tag_name('span')
      # tagTitle = masReviews[i].find_element_by_class_name('title')
      # tagTextoReview = masReviews[i].find_element_by_class_name("text,show-more__control")

      #estos 3 siguientes son ints
      positivas, totales = obtenerHelpfulness(masHelpfulness[i]) #cant de votos positivos y cant de votos totales que obtuvo esa review
      ratio = 0 #y un ratio de que tan helpfulness es esa review
      if(totales != 0):
        ratio = positivas / totales

      #y estos 3 siguientes son strings
      titulo = tagTitle.get_attribute('textContent').lstrip().rstrip() #descubrí que .get_attribute('textContent') es con lo que obtengo el str del WebElement
      puntaje = int(tagScore[0].get_attribute('textContent'))
      texto = tagTextoReview.get_attribute('textContent').lstrip().rstrip()

      #y apendeo eso a las listas que creé arriba
      titulos.append(titulo)
      puntajes.append(puntaje)
      textos.append(texto)
      ratios.append(ratio)
      vPos.append(positivas)
      vTotales.append(totales)

    except:
      pass

  #y con esas listas hago el DF
  data = { 'titulos': titulos, 'puntajes': puntajes, 'textos': textos, 'ratio': ratios, 'positivas': vPos, 'totales': vTotales }
  df = pd.DataFrame.from_dict(data)

  return df

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#food for thoughts: deberia hacer que esté balanceados? o entreno al modelo con todos?
#OJO: que comenté hacer el click en load more
#OJOx222: está limitado en hazDFdeReviews, cuántas reviews veo en total
#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
urlInicio = 'https://www.imdb.com/title/tt'
dfsLista = []

for id in las10MasRecientesYMasPopulares[14:]:
  #inicio scrapping haciendo un webdriver (un explorador) en la id de la peli que me interesa
  url = urlInicio + id + '/reviews'
  driver = webdriver.Chrome('chromedriver',options = options)
  driver.get(url)

  #estas 3 lineas son para mi de ansiosa para saber que empezó a cargar las reviews de una nueva película y cuantas son en total
  try:
    header = driver.find_elements(By.CLASS_NAME, "header") #obtengo header
    cantReviews = int(header[1].get_attribute('textContent')[9:14].replace(',', ''))
    print('Hay: ' + str(cantReviews) + ' reviews')
  except:
    header = driver.find_elements(By.CLASS_NAME, "header") #obtengo header
    cantReviews = int(header[1].get_attribute('textContent')[9:13].replace(',', ''))
    print('Hay: ' + str(cantReviews) + ' reviews')

  #si tengo la cant de reviews en total y sé que 1) la página ya muestra 25 al cargarla y 2) cada vez que presiono en 'load more' va mostrando de a 25 reviews
  #cuento cuantas veces debo hacer click para que mi explorador cargue el total de las reviews
  cantDeVecesQueDeboHacerClick = (cantReviews - 25) // 25

  #en este ciclo tarda porque está cargando tooOoODas las reviews que tiene esa peli
  i=0
  while i < cantDeVecesQueDeboHacerClick:
    try:
        report1 = driver.find_element(By.XPATH, "//button[@id='load-more-trigger']")
        report1.click()
        i+=1
    except:
        pass

  #y guardo las etiquetas que contienen la info que me importa de cada review: "puntaje, title, textoDeReview, ratioHelpfulness"
  masReviews = driver.find_elements(By.CLASS_NAME, 'review-container')
  masHelpfulness = driver.find_elements(By.CLASS_NAME, 'actions,text-muted')

  dfTemp = hazDFdeReviews(masReviews, masHelpfulness)

  #Y luego ese DF lo convierto a .csv y lo guarda en los archivos (hay que darle al icono a la izq del colab con una carpetita)
  dfTemp.to_csv(str(id) + '.csv') #con estos archivos luego ya se pueden trabajar tranquilamente en otro colab sin tener que cargar todas las reviews a cada rato

  dfsLista.append(dfTemp)
  driver.quit()

Hay: 4059 reviews
Va el 25% de las reviews cargadas
Va el 50% de las reviews cargadas
Va el 75% de las reviews cargadas
Va el 95% de las reviews cargadas


In [ ]:
//Hay: 4059 reviews
2378+2713+1446+7230+1334+1034+5432+5997+3557+5569

36690

In [ ]:
dfOrdenado.iloc[indicesPopulares]

,pelisIDs,nombres,dia,mes,year,votos,posters
20,1745960,Top Gun: Maverick,27,5,2022,177326,https://m.media-amazon.com/images/M/MV5BMmIwZD...
27,9419884,Doctor Strange in the Multiverse of Madness,6,5,2022,210219,https://m.media-amazon.com/images/M/MV5BNWM0ZG...
36,6710474,Everything Everywhere All at Once,25,3,2022,120351,https://m.media-amazon.com/images/M/MV5BYTdiOT...
40,1877830,The Batman,4,3,2022,517559,https://m.media-amazon.com/images/M/MV5BMDdmMT...
41,1464335,Uncharted,18,2,2022,135108,https://m.media-amazon.com/images/M/MV5BMWEwNj...
42,7657566,Death on the Nile,11,2,2022,118633,https://m.media-amazon.com/images/M/MV5BNjI4ZT...
46,10838180,The Matrix Resurrections,22,12,2021,231580,https://m.media-amazon.com/images/M/MV5BMGJkND...
47,10872600,Spider-Man: No Way Home,17,12,2021,658596,https://m.media-amazon.com/images/M/MV5BZWMyYz...
48,9032400,Eternals,5,11,2021,313045,https://m.media-amazon.com/images/M/MV5BYzVjYT...
49,1160419,Dune,22,10,2021,581926,https://m.media-amazon.com/images/M/MV5BN2FjNm...


In [ ]:
i=0

print(dfOrdenado['nombres'].iloc[indicesPopulares[i]])
dfsLista[i]

Top Gun: Maverick


,titulos,puntajes,textos,ratio,positivas,totales
0,The truly epic blockbuster we needed.,10,"Wow. The first Top Gun is a classic, and as we...",0.919045,1578,1717
1,This is slightly different to the other review...,10,If you were a late teen or in your early twent...,0.965493,2798,2898
2,Best Sequel yet,10,In one of the more memorable lines in the orig...,0.882732,685,776
3,This is why we go to the movies,10,This is one sequel that looked like it would n...,0.880077,455,517
4,The real cinema experience!,10,If there's any movie that deserves to be seen ...,0.906494,349,385
5,Let me just say...,10,"I was reluctantly dragged into the theater, th...",0.916763,793,865
6,A perfect gift to all Top Gun fans,10,There are so many references to the original t...,0.895425,137,153
7,Flying High,10,"Top Gun (1986) made Tom Cruise a star, and now...",0.820580,311,379
8,What an excellent sequel,9,"What an excellent sequel - I, in fact, like it...",0.814721,321,394
9,Talk to me Goose,10,It makes more than sense to have seen the firs...,0.797571,197,247
